In [ ]:
import time

In [ ]:
import json

In [ ]:
from datetime import datetime

In [ ]:
import nexussdk as nxs

In [ ]:
# Example:
# https://nexus.domain.org/v1
with open("DEPLOYMENT", "r") as f:
    DEPLOYMENT = f.readline().rstrip()
    nxs.config.set_environment(DEPLOYMENT)

In [ ]:
with open("TOKEN", "r") as f:
    TOKEN = f.readline().rstrip()
    nxs.config.set_token(TOKEN)

In [ ]:
pp = nxs.tools.pretty_print

### Examples setup

In [ ]:
def utcnow() -> str:
    return datetime.utcnow().strftime("%Y%m%d%H%M%S%f")

In [ ]:
def generate_compacted_id() -> str:
    return "nxv:myview{}".format(utcnow())

In [ ]:
def generate_expanded_id() -> str:
    return "{}/myview{}".format("https://knowledgegraph.domain.org", utcnow())

In [ ]:
ORGANIZATION = "myorg{}".format(utcnow())

In [ ]:
nxs.organizations.create(ORGANIZATION)["@id"]

In [ ]:
PROJECT = "myproject"

In [ ]:
nxs.projects.create(ORGANIZATION, PROJECT)["@id"]

In [ ]:
PROJECT2 = "{}/myproject2".format(ORGANIZATION)

In [ ]:
nxs.projects.create(*PROJECT2.split("/"))["@id"]

In [ ]:
project3 = "{}/myproject3".format(ORGANIZATION)

In [ ]:
nxs.projects.create(*project3.split("/"))["@id"]

In [ ]:
projects = [PROJECT2, project3]

In [ ]:
mappings = {
    "doc": {
      "properties": {
        "@id": {
          "type": "keyword"
        },
        "_createdAt":  {
            "type": "date"
        },
      }
    }
}

### User API

In [ ]:
es_view1 = nxs.views.create_es(ORGANIZATION, PROJECT, mappings)


In [ ]:
es_view2 = nxs.views.create_es(ORGANIZATION, PROJECT, mappings, tag="test")

In [ ]:
es_view3 = nxs.views.create_es(ORGANIZATION, PROJECT, mappings,
                               resource_schemas = ["https://bluebrain.github.io/nexus/schemas/view.json"],
                               resource_types = ["https://bluebrain.github.io/nexus/schemas/view.json"],
                               source_as_text = True, include_metadata = True, include_deprecated = True)

In [ ]:
pp(es_view3)

In [ ]:
agg_es_view = nxs.views.create_es_aggregated(ORGANIZATION, PROJECT, [es_view1, es_view2, es_view3], generate_compacted_id())

In [ ]:
sparql_view1 = nxs.views.create_sparql(ORGANIZATION, PROJECT)

In [ ]:
sparql_view2 = nxs.views.create_sparql(ORGANIZATION, PROJECT, tag="test")

In [ ]:
sparql_view3 = nxs.views.create_sparql(ORGANIZATION, PROJECT,
                                       resource_schemas = ["https://bluebrain.github.io/nexus/schemas/view.json"],
                                       resource_types = ["https://bluebrain.github.io/nexus/schemas/view.json"],
                                       include_metadata = True, include_deprecated = True)

In [ ]:
agg_sparql_view =  nxs.views.create_sparql_aggregated(ORGANIZATION, PROJECT, [sparql_view1, sparql_view2, sparql_view3], generate_compacted_id())

In [ ]:
es_query = {
    "query": {
        "match_all": {}
    }
}

In [ ]:
pp(nxs.views.query_es(ORGANIZATION, PROJECT, es_query))

In [ ]:
pp(nxs.views.query_es(ORGANIZATION, PROJECT,es_query, es_view1["@id"] ))

In [ ]:
pp(nxs.views.query_es(ORGANIZATION, PROJECT,es_query, agg_es_view["@id"] ))

In [ ]:
sparql_query = "select ?s ?o where {  ?s rdf:type ?o . }"

In [ ]:
pp(nxs.views.query_sparql(ORGANIZATION, PROJECT,sparql_query))

In [ ]:
pp(nxs.views.query_sparql(ORGANIZATION, PROJECT,sparql_query, sparql_view1["@id"]))

In [ ]:
pp(nxs.views.query_sparql(ORGANIZATION, PROJECT,sparql_query, agg_sparql_view["@id"]))

In [ ]:
pp(nxs.views.stats(ORGANIZATION, PROJECT, sparql_view1["@id"]))